In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## Tensorflow Probability: spatial patterns in genetics

This is using data generated in the `spatial_population_structure_genetic_simulations` notebook.

In [2]:
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np

tfd = tf.contrib.distributions

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/usr/local/lib/python3.5/dist-packages/tensorflow/python/util/tf_inspect.py:66: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  _inspect.getargspec(target))
/usr/local/lib/python3.5/dist-packages/tensorflow/python/util/tf_inspect.py:66: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  _inspect.getargspec(target))
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility

/usr/local/lib/python3.5/dist-packages/tensorflow/python/util/tf_inspect.py:66: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  _inspect.getargspec(target))
/usr/local/lib/python3.5/dist-packages/tensorflow/python/util/tf_inspect.py:66: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  _inspect.getargspec(target))
/usr/local/lib/python3.5/dist-packages/tensorflow/python/util/tf_inspect.py:66: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  _inspect.getargspec(target))
/usr/local/lib/python3.5/dist-packages/tensorflow/python/util/tf_inspect.py:66: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  _inspect.getargspec(target))
/usr/local/lib/python3.5/dist-packages/tensorflow/python/util/tf_inspect.py:66: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  _inspect.getargspec(target))
/usr/

In [3]:
# simulation parameters copied from other notebook.
n = 10000 # individuals
m = 1000 # variants
d = 3 # latent dimension

# estiamtion routine parameters
batch_size = 100
epochs = 10

X = np.load('./data/spatial_a-0.1_N-'+ str(n) + '_M-' + str(m) +'.pandas.gz.npy')
X = X.astype(np.float32)

### Basic VAE

We'll start with a basic, deep, variational autoencoder. Our goal is to try and recover something similar to our specified latent dimension. We will test out how models are specified in `tfp`!

This is more-or-less following the example given in the official repository: https://github.com/tensorflow/probability/blob/master/tensorflow_probability/examples/vae.py

In [4]:
def make_encoder(x, latent_dimension=2):
    x = tf.layers.dense(inputs=x,
            units=512, activation=tf.nn.sigmoid)
    x = tf.layers.dense(inputs=x,
            units=256, activation=tf.nn.sigmoid)
    x = tf.layers.dense(inputs=x,
            units=128, activation=tf.nn.sigmoid)
    encoder_net = tf.layers.dense(inputs=x,
                      units = latent_dimension * 2,
                      activation=None)
    
    loc = encoder_net[..., :latent_dimension]
    scale = tf.nn.softplus(encoder_net[..., latent_dimension:] + 0.5)
    return tfd.MultivariateNormalDiag(loc=loc,
            scale_diag=scale,
            name="encoder_distribution")
    

def make_decoder(latent_code, num_features=m):
    x = tf.layers.dense(inputs=latent_code,
            units=128, activation=tf.nn.sigmoid)
    x = tf.layers.dense(inputs=x,
             units=256, activation=tf.nn.sigmoid)
    x = tf.layers.dense(inputs=x,
            units=512, activation=tf.nn.sigmoid)
    
    decoder_net = tf.layers.dense(inputs=x,
            units=num_features, activation=None)    
    
    return tfd.Independent(tfd.Binomial(logits=decoder_net,
                            total_count=2.0),
                            reinterpreted_batch_ndims=1,
                            name="decoder_distribution")

def make_prior(latent_dimension):
    prior =  tfd.MultivariateNormalDiag(scale_diag=tf.ones(latent_dimension),
                                    name="prior_distribution")
    return prior


In [5]:
graph = tf.get_default_graph()
with graph.as_default():
    # input pipeline
    dataset = tf.data.Dataset.from_tensor_slices(X)
    dataset = dataset.repeat(epochs)
    dataset = dataset.batch(batch_size)
    iterator = dataset.make_initializable_iterator()
    data = iterator.get_next()
    
    x = tf.reshape(data, [batch_size, m, 1])
        
    # inference network; encoder
    with tf.variable_scope('encoder'):
        encoder = make_encoder(x)

    # prior
    with tf.variable_scope('prior'):
        prior = make_prior(latent_dimension=2)

    # loss
    def joint_log_prob(z):
        with tf.variable_scope('decoder'):
            decoder = make_decoder(z)
        return decoder.log_prob(x) + prior.log_prob(z)

    
    elbo = tf.reduce_sum(
            tfp.vi.monte_carlo_csiszar_f_divergence(
                f=tfp.vi.kl_reverse,
                p_log_prob=joint_log_prob,
                q=encoder,
                num_draws=1))
    
    # optimizer
    optimizer = tf.train.AdamOptimizer(0.001).minimize(elbo)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [6]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(iterator.initializer)
    
    for epoch in range(epochs):
        _, elbo = sess.run([optimizer, elbo])
        print(elbo)

118965170.0


TypeError: Fetch argument 118965170.0 has invalid type <class 'numpy.float32'>, must be a string or Tensor. (Can not convert a float32 into a Tensor or Operation.)